In [1]:
from typing import List, Optional, Generator
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect
import heapq

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str, dtypes: dict[str, str]=dict()) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    temp = temp.astype(dtypes)
    return temp

conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)

        while visited and not visited[-1]:
            visited.pop()

        return visited

    def __str__(self):
        return str(self.val)

    @staticmethod
    def from_array(nums: list[int|None]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)

                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    @staticmethod
    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr

        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

#### Utility to generate random BST

In [4]:
def generateBST(N: int, min_: int, max_: int) -> BinaryTreeNode|None:
    def insert(curr: BinaryTreeNode|None, n: int) -> BinaryTreeNode:
        if not curr:
            curr = BinaryTreeNode(n)
        elif curr.val < n:
            curr.right = insert(curr.right, n)
        else:
            curr.left = insert(curr.left, n)

        return curr

    assert N <= max_ - min_, "Number of available samples must be >= N"
    root: BinaryTreeNode|None = None
    for n in np.random.choice(np.arange(min_, max_), size=N, replace=False):
        root = insert(root, n)

    return root

Best time to buy and sell stocks - I

In [5]:
def stocks1(prices: list[int]) -> int:
    min_, max_profit = prices[0], 0
    for price in prices:
        max_profit = max(max_profit, price - min_)
        min_ = min(min_, price)

    return max_profit

# Testing the solution
assert stocks1([7,1,5,3,6,4]) == 5
assert stocks1([7,6,4,3,1]) == 0

Weekly Contest 398: 19th May 2024

In [6]:
# Q1
def isArraySpecial(nums: list[int]) -> bool:
    N = len(nums)
    for i in range(N - 1):
        i1, i2 = nums[i] % 2, nums[i + 1] % 2
        if (i1 == 0 and i2 == 0) or (i1 == 1 and i2 == 1):
            return False
    else:
        return True

# Testing the solution
assert isArraySpecial([4,3,1,6]) == False
assert isArraySpecial([2,1,4]) == True
assert isArraySpecial([1]) == True

In [7]:
# Q2
def isArraySpecial2(nums: list[int], queries: list[list[int]]) -> list[bool]:
    N = len(nums)
    parity: list[int] = list(map(lambda x: x % 2, nums))

    @functools.cache
    def backtrack(start: int, end: int) -> bool:
        if start == end:
            return True
        else:
            mid = (start + end) // 2
            isSpecial =  parity[mid] != parity[mid + 1] and backtrack(start, mid) and backtrack(mid + 1, end)
            return isSpecial

    result: list[bool] = []
    for i, j in queries:
        result.append(backtrack(i, j))

    return result

# Testing the solution
assert isArraySpecial2([4,3,1,6], [[0,2],[2,3]]) == [False, True]
assert isArraySpecial2([3,4,1,2,6], [[0,4]]) == [False]

In [8]:
# Q3
def sumDigitDifferencesBrute(nums: list[int]) -> int:
    length = len(str(nums[0]))
    count = 0
    for n1, n2 in itertools.combinations(nums, r=2):
        n1_str, n2_str = str(n1), str(n2)
        for i in range(length):
            if n1_str[i] != n2_str[i]:
                count += 1

    return count

def sumDigitDifferences(nums: list[int]) -> int:
    def count(arr: list[int]) -> int:
        N = len(arr)
        temp = 0
        for i in range(N - 1):
            temp += arr[i] * sum(arr[i + 1:])

        return temp

    length = len(str(nums[0]))
    digits_dict: dict[int, dict[str, int]] = dict()
    for n in nums:
        n_str = str(n)
        for i in range(length):
            positions = digits_dict.get(i, dict())
            positions[n_str[i]] = positions.get(n_str[i], 0) + 1
            digits_dict[i] = positions

    result: int = 0
    for i in range(length):
        result += count(list(digits_dict[i].values()))

    return result

# Testing the solution
for i in range(100):
    temp = list(np.random.randint(10, 100, 50))
    assert sumDigitDifferencesBrute(temp) == sumDigitDifferences(temp)

Sum of XOR totals
Easy LC: https://leetcode.com/problems/sum-of-all-subset-xor-totals

In [9]:
# https://leetcode.com/problems/sum-of-all-subset-xor-totals/submissions/1263288868
def subsetXORSumMemo(nums: list[int]) -> int:
    @functools.cache
    def backtrack(i: int, curr: int) -> int:
        if i == N:
            return curr
        else:
            return backtrack(i + 1, curr ^ nums[i]) + backtrack(i + 1, curr)

    N = len(nums)
    return backtrack(0, 0)

# Testing the solution
assert subsetXORSumMemo([1,3]) == 6
assert subsetXORSumMemo([5,1,6]) == 28

Largest Rectangle in a histogram

In [10]:
# https://leetcode.com/problems/largest-rectangle-in-histogram/submissions/1266278725
def largestRectangleArea(heights: list[int]) -> int:
    """
    Count the number of rectangles to the right that are greater than or equal to self
    Count the number of rectangles to the left that are greater than or equal to self
    """

    N = len(heights)

    right_dp: list[int] = [N - i for i in range(N)]
    left_dp: list[int] = [i + 1 for i in range(N)]
    stack: list[tuple[int, int]] = []

    for i in range(N):
        curr: tuple[int, int] = (heights[i], i)
        while stack and stack[-1][0] > curr[0]:
            prev = stack.pop()
            right_dp[prev[1]] = curr[1] - prev[1]
        stack.append(curr)

    stack.clear()

    for i in range(N - 1, -1, -1):
        curr = (heights[i], i)
        while stack and stack[-1][0] > curr[0]:
            prev = stack.pop()
            left_dp[prev[1]] = prev[1] - curr[1]
        stack.append(curr)

    max_ = 0
    for i in range(N):
        max_ = max(max_, (left_dp[i] + right_dp[i] - 1) * heights[i])

    return max_

# Testing the solution
assert largestRectangleArea([2,1,5,6,2,3]) == 10
assert largestRectangleArea([2,4]) == 4
assert largestRectangleArea([2,1,2]) == 3

Container Queries - CN Weekly - 3

In [11]:
def containerQueries(N: int, S: str, query_len: int, queries: list[list[int]]) -> list[int]:
    positions: dict[str, collections.deque[int]] = dict()
    for i in range(N):
        indices = positions.get(S[i], collections.deque())
        indices.append(i)
        positions[S[i]] = indices

    end_pos = 0
    for k in positions:
        end_pos = max(end_pos, positions[k].popleft())

    dp: list[int] = [end_pos]
    for i in range(1, N):
        prev = S[i - 1]
        next_pos = max(dp[-1], positions[prev].popleft() if positions[prev] else N)
        dp.append(next_pos)

    result: list[int] = []
    for i, j in queries:
        if j >= dp[i]:
            result.append(1)
        else:
            result.append(0)

    return result

# Testing the solution
assert containerQueries(8, "yukppuyk", 2, [[0,4],[5,7]]) == [1, 0]

LC Biweekly 131: 25th May 2024

In [12]:
# Q1
def duplicateNumbersXOR(nums: list[int]) -> int:
    set_: set[int] = set()
    result: int = 0
    for n in nums:
        if n not in set_:
            set_.add(n)
        else:
            result = result ^ n

    return result

# Testing the solution
assert duplicateNumbersXOR([1,2,1,3]) == 1
assert duplicateNumbersXOR([1,2,3]) == 0
assert duplicateNumbersXOR([1,2,2,1]) == 3

In [13]:
# Q2
def occurrencesOfElement(nums: list[int], queries: list[int], x: int) -> list[int]:
    N = len(nums)
    occurance: int = 0
    indices: dict[int, int] = dict()
    for i in range(N):
        if nums[i] == x:
            indices[occurance] = i
            occurance += 1

    result: list[int] = []
    for q in queries:
        result.append(indices.get(q - 1, -1))

    return result

# Testing the solution
assert occurrencesOfElement([1,1,1], [1,2,3,4], 1) == [0,1,2,-1]
assert occurrencesOfElement([1,3,1,7], [1,3,2,4], 1) == [0,-1,2,-1]
assert occurrencesOfElement([1,2,3], [10], 5) == [-1]

In [14]:
# Q3
def queryResults(limit: int, queries: list[list[int]]) -> list[int]:
    color_map: dict[int, set[int]] = dict()
    ball_map: dict[int, int] = dict()

    result: list[int] = []
    for ball, color in queries:
        old_color = ball_map.get(ball, 0)
        if old_color > 0:
            color_map[old_color].remove(ball)
            if len(color_map[old_color]) == 0:
                del color_map[old_color]

        balls = color_map.get(color, set())
        balls.add(ball)
        color_map[color] = balls
        ball_map[ball] = color

        result.append(len(color_map))

    return result

# Testing the solution
assert queryResults(4, [[0,1],[1,2],[2,2],[3,4],[4,5]]) == [1,2,2,3,4]
assert queryResults(4, [[1,4],[2,5],[1,3],[3,4]]) == [1,2,2,3]
assert queryResults(1, [[0,10],[1,10]]) == [1,1]

In [15]:
# Q4: TLE :(
import bisect

def getResultsBrute(queries: list[list[int]]) -> list[bool]:
    obstacles: list[int] = [0]
    result: list[bool] = []

    for q in queries:
        if len(q) == 2:
            bisect.insort(obstacles, q[1])
        else:
            x, sz = q[1], q[2]
            idx = bisect.bisect_left(obstacles, x)
            max_size = x - obstacles[idx - 1]
            for i in range(1, idx):
                max_size = max(max_size, obstacles[i] - obstacles[i - 1])

            result.append(max_size >= sz)

    return result

# Testing the solution
assert getResultsBrute([[1,2],[2,3,3],[2,3,1],[2,2,2]]) == [False, True, True]
assert getResultsBrute([[1,7],[2,7,6],[1,2],[2,7,5],[2,7,6]]) == [True,True,False]

CN Weekly Contest

In [16]:
# Q2 - Easy
def minimumLength(s1: str, s2: str) -> int:
    N, M = len(s1), len(s2)

    if s1 in s2:
        return M

    elif s2 in s1:
        return N

    else:
        # Store the indices where s1[0] and s1[-1] are found
        left: list[int] = []
        right: list[int] = []
        for i in range(M):
            if s2[i] == s1[0]:
                right.append(i)
            if s2[i] == s1[-1]:
                left.append(i)

        # Ans is always possible
        min_ = N + M

        # Check right concatentation
        for i in right:
            overlap_length = M - i
            if s2[i:] == s1[:overlap_length]:
                min_ = min(min_, N + M - overlap_length)

        # Check left concatentation
        for i in left:
            overlap_length = i + 1
            if s2[:i + 1] == s1[-overlap_length:]:
                min_ = min(min_, N + M - overlap_length)

        return min_

# Testing the solution
assert minimumLength("abcabc", "aaabc") == 8
assert minimumLength("bibia", "aibia") == 9
assert minimumLength("aabcabcd", "abc") == 8

LC Medium: Count number of primes
https://leetcode.com/problems/count-primes/

In [17]:
# https://leetcode.com/problems/count-primes/submissions/1275257285
def countPrimes(N: int) -> int:

    if N == 0:
        return 0

    soe: list[bool] = [True] * (N + 1)
    soe[0] = soe[1] = False
    i = 2
    while i * i <= N:
        if soe[i]:
            """
            j = i
            while i * j <= N:
                soe[i * j] = False
                j += 1
            """
            # List slicing is much faster thanks to the C impl backend
            soe[i*i:N+1:i] = [False] * len(soe[i*i:N+1:i])
        i += 1

    return sum(soe)

# Testing the solution
assert countPrimes(10) == 4
assert countPrimes(0) == countPrimes(1) == 0
assert countPrimes(100) == 25

Heights checker - Easy
https://leetcode.com/problems/height-checker/submissions/1283296449/

In [18]:
def heightChecker(heights: list[int]) -> int:
    """
    Implement count sort
    Time: O(N), Space: O(N)

    1. Bucketize and count the heights
    2. Kind of like a two pointer approach, iterate through both. Bucketized dict is iterate in order of natural numbers O(100 * individual freq) ~ O(N)
    3. This solution is O(N) only because we have a natural limit to max height - 100
    """
    freq = collections.Counter(heights)

    result = currHeight = 0
    for height in heights:

        while currHeight not in freq or freq[currHeight] == 0: 
            currHeight += 1

        if currHeight != height:
            result += 1

        # Whether matching or not matching, one of student's
        # heights were compared, so continue travesing
        freq[currHeight] -= 1

    return result

# Testing the solution
assert heightChecker([1,1,4,2,1,3]) == 3
assert heightChecker([5,1,2,3,4]) == 5
assert heightChecker([1,2,3,4,5]) == 0

LC Virtual - 1: Warmup contest
https://leetcode.com/contest/warm-up-contest

In [19]:
# Q1: Lexicographical numbers
def lexicalOrderRecursive(n: int) -> list[int]:
    result: list[int] = []
    def backtrack(curr: int) -> None:
        result.append(curr)
        if curr * 10 <= n:
            backtrack(curr * 10)
        if (curr % 10) < 9 and curr < n:
            backtrack(curr + 1)

    backtrack(1)
    return result

# Testing the solution
lexicalOrderRecursive(25)

[1,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 2,
 20,
 21,
 22,
 23,
 24,
 25,
 3,
 4,
 5,
 6,
 7,
 8,
 9]

In [20]:
def lexicalOrder(n: int) -> list[int]:
    result: list[int] = []
    stack: list[int] = [1]
    while stack:
        curr = stack.pop()
        result.append(curr)
        if (curr % 10) < 9 and curr < n:
            stack.append(curr + 1)
        if curr * 10 <= n:
            stack.append(curr * 10)

    return result

# Testing the solution
lexicalOrder(25)

[1,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 2,
 20,
 21,
 22,
 23,
 24,
 25,
 3,
 4,
 5,
 6,
 7,
 8,
 9]

In [21]:
# Q2: First unique character in a string
def firstUniqChar(s: str) -> int:
    """
    Iterate from right to left. Keep track of elements already visited.
    If new char is encountered update result.
    Else (char already exists), ensure that result prev found
    """
    N = len(s)
    freq = collections.Counter(s)
    for i in range(N):
        if freq[s[i]] == 1:
            return i
    return -1

# Testing the solution
assert firstUniqChar("leetcode") == 0
assert firstUniqChar("loveleetcode") == 2
assert firstUniqChar("aabb") == -1
assert firstUniqChar("aab") == 2

In [22]:
# Q3: Longest Absolute File Path
def lengthLongestPath(abstracted: str) -> int:
    """
    Iterate through path character by character.
    "\n" demarcates the structure.
    "\t" tells us about the depth

    If curr_depth is > prev_depth, nest inside previous.
    If curr_depth <= prev_depth, remove previously added levels and add curr. Levels to remove: prev_depth - curr_depth + 1
    """

    path: list[str] = []
    N, i = len(abstracted), 0
    max_length, depth = 0, 0
    is_file = False
    while i < N:
        if abstracted[i] == "\n":
            # Check if current absolute path is a file, if yes update max_length
            if is_file:
                max_length = max(max_length, len(path))

            i, prev_depth, depth = i + 1, depth, 0
            while abstracted[i] == "\t":
                i, depth = i + 1, depth + 1

            # Remove nested structure if curr_depth <= prev_depth
            if depth <= prev_depth:
                extra_depth = prev_depth - depth + 1
                while path and extra_depth:
                    removed = path.pop()
                    if removed == "/":
                        extra_depth -= 1

            # Reset values for upcoming cycle
            if depth > 0:
                path.append("/")
            is_file = False

        else:
            if abstracted[i] == '.':
                is_file = True
            path.append(abstracted[i])
            i += 1

    return max(max_length, len(path) if is_file else 0)

# Testing the solution
assert lengthLongestPath("dir\n\tsubdir1\n\t\tfile1.ext\n\t\tsubsubdir1\n\tsubdir2\n\t\tsubsubdir2\n\t\t\tfile2.ext") == 32
assert lengthLongestPath("dir\n\tsubdir1\n\tsubdir2\n\t\tfile.ext") == 20
assert lengthLongestPath("file1.txt\nfile2.txt\nlongfile.txt") == 12
assert lengthLongestPath("dir1\ndir2\n\tsubdir1\n\tsubdir2\n\t\tfile.ext") == 21
assert lengthLongestPath("dir0\n\tabcdefghijklmnopqrstuvw.file") == 33

LC Medium: Furthest Building you can reach
https://leetcode.com/problems/furthest-building-you-can-reach/submissions/1284332700/

In [23]:
def furthestBuilding(heights: list[int], bricks: int, ladders: int) -> int:
    """
    Maintain the sum of deltas, maintain a max heap of size ladders.
    At any point total sum - heap sum > bricks, break
    """
    class MinHeap:
        def __init__(self, size: int):
            self.capacity: int = size
            self.heap: list[int] = []
            self.sum: int = 0
            self.N = 0

        def add(self, delta: int) -> int:
            """
            Adds to the heap and returns the sum of the heap
            """
            if self.N < self.capacity:
                heapq.heappush(self.heap, delta)
                self.sum, self.N = self.sum + delta, self.N + 1
            else:
                # Equivalent to a push, then a pop
                extra = heapq.heappushpop(self.heap, delta)
                self.sum += delta - extra

            return self.sum

        def __str__(self) -> str:
            return ','.join(sorted(map(str, self.heap)))

    N = len(heights)
    heap = MinHeap(ladders)
    heap.add(0)
    total_sum, prev = 0, heights[0]
    for i in range(N):
        prev, delta = heights[i], heights[i] - prev
        if delta >= 0:
            total_sum += delta
            heap_sum = heap.add(delta)
            if total_sum - heap_sum > bricks:
                return i - 1

    return i

# Testing the solution
assert furthestBuilding([4,2,7,6,9,14,12], 5, 1) == 4
assert furthestBuilding([4,12,2,7,3,18,20,3,19], 10, 2) == 7
assert furthestBuilding([14,3,19,3], 17, 0) == 3
assert furthestBuilding([1,5,1,2,3,4,10000], 4, 1) == 5

LC Virtual - 3: Weekly Contest 2
https://leetcode.com/contest/leetcode-weekly-contest-2

In [24]:
# LC Easy: Q1: Find the difference
def findTheDifference(s: str, t: str) -> str:
    s_freq = collections.Counter(s)
    for ch in t:
        if ch not in s_freq:
            return ch
        else:
            s_freq[ch] -= 1
            if s_freq[ch] == 0:
                s_freq.pop(ch)

    return ""

# Testing the solution
assert findTheDifference("ab", "aba") == 'a'
assert findTheDifference("", "x") == 'x'

In [25]:
# LC Medium: Q2: Elimination Game
def lastRemainingBrute(N: int) -> int:
    not_eliminated: set[int] = set(range(1, N + 1))
    skip, dir_ = 1, 1
    while len(not_eliminated) >  1:
        # RTL
        if dir_ == 1:
            start = min(not_eliminated)
            end, skip, dir_ = N + 1, 2 * abs(skip), -1

        # LTR
        else:
            start = max(not_eliminated)
            end, skip, dir_ = 0, -2 * abs(skip), 1

        print(not_eliminated)

        for i in range(start, end, skip):
            not_eliminated.remove(i)

    return not_eliminated.pop()

# Testing the solution
assert lastRemainingBrute(9) == 6

{1, 2, 3, 4, 5, 6, 7, 8, 9}
{2, 4, 6, 8}
{2, 6}


In [26]:
def lastRemaining(N: int) -> int:
    """
    Binary Search like approach:
        If N is odd: Shrink both ends
        Elif dir == "right": Shrink to right
        Else: Shrink to left
    """
    start, end = 1, N
    range_length, shrink_size = N, 1
    rtl = True
    while range_length > 1:
        if range_length % 2:
            start, end = start + shrink_size, end - shrink_size
        elif rtl:
            start = start + shrink_size
        else:
            end = end - shrink_size

        rtl = not rtl
        range_length, shrink_size = range_length // 2, shrink_size * 2

    return start

# Testing the solution
assert lastRemaining(9) == 6
assert lastRemaining(1) == 1

LC Easy: Rectangle Overlap

In [ ]:
def isRectangleOverlap(rec1: list[int], rec2: list[int]) -> bool:
    # Store to variables for clarity
    (r1_x1, r1_y1, r1_x2, r1_y2) = rec1
    (r2_x1, r2_y1, r2_x2, r2_y2) = rec2

    # If one rectangle is on the left side of the other
    if r1_x2 < r2_x1 or r2_x2 < r1_x1:
        return False

    # If one rectangle is above the other
    elif r1_y2 < r2_y1 or r2_y2 < r1_y1:
        return False

    # They must overlap
    else:
        return True

LC Hard: Perfect Rectangle

In [ ]:
def isRectangleCoverBrute(rectangles: list[list[int]]) -> bool:
    """
    Time: O(N ** 2), Space: O(1)
    """
    def isOverlap(r1: list[int], r2: list[int]) -> bool:
        """
        Time: O(1)
        """
        r1_x1, r1_y1, r1_x2, r1_y2 = r1
        r2_x1, r2_y1, r2_x2, r2_y2 = r2

        if r1_x2 <= r2_x1 or r2_x2 <= r1_x1:
            return False
        elif r1_y2 <= r2_y1 or r2_y2 <= r1_y1:
            return False
        else:
            return True

    # Number of rectangles
    N = len(rectangles)

    # Check for overlap
    for i in range(N):
        for j in range(i + 1, N):
            if isOverlap(rectangles[i], rectangles[j]):
                return False

    # Check if area matches
    min_x, min_y, max_x, max_y = math.inf, math.inf, -math.inf, -math.inf
    total_area = 0
    for i in range(N):
        (x1, y1, x2, y2) = rectangles[i]
        min_x = min(min_x, x1, x2)
        min_y = min(min_y, y1, y2)
        max_x = max(max_x, x1, x2)
        max_y = max(max_y, y1, y2)
        total_area += abs(x1 - x2) * abs(y1 - y2)

    return total_area == (max_x - min_x) * (max_y - min_y)

# Testing the solution
assert isRectangleCoverBrute([[1,1,3,3],[3,1,4,2],[1,3,2,4],[2,2,4,4]]) == False
assert isRectangleCoverBrute([[1,1,2,3],[1,3,2,4],[3,1,4,2],[3,2,4,4]]) == False
assert isRectangleCoverBrute([[1,1,3,3],[3,1,4,2],[3,2,4,4],[1,3,2,4],[2,3,3,4]]) == True
assert isRectangleCoverBrute([[0,0,1,1],[0,1,3,2],[1,0,2,2]]) == False
assert isRectangleCoverBrute([[0,0,2,2],[1,1,3,3],[2,0,3,1],[0,3,3,4]]) == False

In [ ]:
# Logic copied from `https://leetcode.com/problems/perfect-rectangle/solutions/2992766/easy-c-solution-o-n-using-map/`
# https://leetcode.com/problems/perfect-rectangle/submissions/1284785419/
def isRectangleCover(rectangles: list[list[int]]) -> bool:
    """
    - Each internal corner (where rectangles meet but do not form an external corner of the overall rectangle) will be counted an even number of times and should cancel out to zero.
    - Each external corner of the overall covering rectangle should appear exactly once.
    - Coordinate counts must either cancel out or equal 0

    Time: O(N), Space: O(N)
    """
    coord_freq: dict[tuple[int, int], int] = dict()
    for rect in rectangles:
        # Tuple unpacking
        x1, y1, x2, y2 = rect

        # Top left, Top Right, Bottom Left, Bottom Right
        tl, tr, bl, br = (x1, y2), (x2, y2), (x1, y1), (x2, y1)

        # Add bottom left, top right
        coord_freq[bl] = coord_freq.get(bl, 0) + 1
        coord_freq[tr] = coord_freq.get(tr, 0) + 1

        # Subtract top left, bottom right
        coord_freq[tl] = coord_freq.get(tl, 0) - 1
        coord_freq[br] = coord_freq.get(br, 0) - 1

    # Ensure that we have exactly 4 corners with count == 1
    corner_count = 0
    for freq in coord_freq.values():
        if abs(freq) == 1:
            corner_count += 1
        elif freq != 0:
            return False

    return corner_count == 4

# Testing the solution
assert isRectangleCover([[1,1,3,3],[3,1,4,2],[1,3,2,4],[2,2,4,4]]) == False
assert isRectangleCover([[1,1,2,3],[1,3,2,4],[3,1,4,2],[3,2,4,4]]) == False
assert isRectangleCover([[1,1,3,3],[3,1,4,2],[3,2,4,4],[1,3,2,4],[2,3,3,4]]) == True
assert isRectangleCover([[0,0,1,1],[0,1,3,2],[1,0,2,2]]) == False
assert isRectangleCover([[0,0,2,2],[1,1,3,3],[2,0,3,1],[0,3,3,4]]) == False

LC Easy: Relative sort array

In [ ]:
def relativeSortArray(arr1: list[int], arr2: list[int]) -> list[int]:
    # Time: O(N2 + N1 log N1), Space: O(N2)
    N1, N2 = len(arr1), len(arr2)
    keys: dict[int, int] = {k: i for i, k in enumerate(arr2)}
    arr1.sort(key=lambda x: keys.get(x, x + N1))
    return arr1

# Testing the solution
assert relativeSortArray([2,3,1,3,2,4,6,7,9,2,19], [2,1,4,3,9,6]) == [2,2,2,1,4,3,3,9,6,7,19]

In [ ]:
def relativeSortArrayBetter(arr1: list[int], arr2: list[int]) -> list[int]:
    """
    1. Count occurances of arr1
    2. Iterate throught arr2 (for preserving order), while freq[n] exists keep appending and decreasing the freq
    3. Sort the left overs and append to the result
    """
    freq: collections.Counter = collections.Counter(arr1)
    result: list[int] = []
    for n in arr2:
        while freq[n]:
            result.append(n)
            freq[n] -= 1
            if freq[n] == 0:
                freq.pop(n)

    for n in sorted(freq.keys()):
        while freq[n]:
            result.append(n)
            freq[n] -= 1

    return result

# Testing the solution
assert relativeSortArrayBetter([2,3,1,3,2,4,6,7,9,2,19], [2,1,4,3,9,6]) == [2,2,2,1,4,3,3,9,6,7,19]

LC Easy: Find Pivot Integer
Runtime comparison
1. C Lang: 0 ms
```C
// To compile run: gcc test.c -o test.out -ln && ./test.out

#include <stdio.h>
#include <math.h>

int pivotInteger(int n) {
    int total = (n * (n + 1)) / 2;    
    int result = sqrt(total);
    return result * result == total? result: -1;
}

int main() {
    int n;
    scanf("%d", &n);
    printf("%d\n", pivotInteger(n));
    return 0;
}
```

2. CPP: 0 ms
```
// To compile run: g++ test.cpp -o test.out && ./test.out

#include <cmath>
#include <iostream>

int pivotInteger(int n) {
    int total = (n * (n + 1)) / 2;
    int result = sqrt(total);
    return result * result == total? result: -1;
}

int main() {
    int n;
    std::cin >> n;
    printf("%d\n", pivotInteger(n));
    return 0;
}
```

In [ ]:
# Runtime: 32 ms
def pivotInteger(n: int) -> int:
    total = (n * (n + 1)) // 2
    result = int(math.sqrt(total))
    return result if result * result == total else -1

# Testing the solution
assert pivotInteger(8) == 6
assert pivotInteger(3) == -1

LC Virtual - 2: Weekly Contest 3
https://leetcode.com/contest/leetcode-weekly-contest-2

In [ ]:
# Q1: Is Subsequence: https://leetcode.com/problems/is-subsequence/
def isSubsequence(s: str, t: str) -> bool:
    """
    If there are a lot of s in sequence to parse, we could try this approach:
        1. Store the indices of each character of t to a hashmap
        2. Have a pointer on both s and t
        3. For each char s[i], check hm[s[i]] for the indices. j is set to smallest index > j. If no such j is available return False right away.
        4. If all elements are done, it is a match.
    """
    N1, N2 = len(s), len(t)
    i = j = 0
    while i < N1 and j < N2:
        if s[i] == t[j]:
            i, j = i + 1, j + 1
        else:
            j += 1
    return i == N1

# Testing the solution
assert isSubsequence("abc", "ahbgdc") == True
assert isSubsequence("axc", "ahbgdc") == False
assert isSubsequence("", "ahbgdc") == True
assert isSubsequence("a", "") == False

In [ ]:
# Q4: Longest Substring with atleast K repeating characters
def longestSubstringBrute(s: str, k: int) -> int:
    N = len(s)
    max_length = 0
    for i in range(N):
        freq: dict[str, int] = dict()
        for j in range(i, N):
            freq[s[j]] = freq.get(s[j], 0) + 1
            if not len(list(filter(lambda x: x < k, freq.values()))):
                max_length = max(max_length, j - i + 1)

    return max_length

# Testing the solution
assert longestSubstringBrute("aaabb", 3) == 3
assert longestSubstringBrute("ababbc", 2) == 5

In [ ]:
# Q3: Decode String
def decodeString(s: str) -> str:
    N = len(s)
    stack: list[str] = []
    for ch in s:
        if ch != "]":
            stack.append(ch)
        else:
            acc: list[str] = []
            while stack[-1] != '[':
                acc.append(stack.pop())

            # Pop out '['
            stack.pop()

            # Reverse accumulated content
            acc.reverse()

            # Multiply and append as new item
            multiplier: list[str] = []
            while stack and stack[-1].isnumeric():
                multiplier.append(stack.pop())
            multiplier.reverse()

            stack.append(int(''.join(multiplier)) * ''.join(acc))

    return ''.join(stack)

# Testing the solution
assert decodeString("3[a2[c]]") == "accaccacc"
assert decodeString("3[a]2[bc]") == "aaabcbc"
assert decodeString("2[abc]3[cd]ef") == "abcabccdcdcdef"
assert decodeString("11[a]") == "aaaaaaaaaaa"

In [ ]:
# Q2: Utf-8 Validation
def validUtf8(data: list[int]) -> bool:
    def num2Bin(n: int) -> str:
        return bin(n)[2:].rjust(8, '0')

    i, N = 0, len(data)
    while i < N:
        curr: str = num2Bin(data[i])
        idx = curr.find('0')
        if idx == 0:
            i += 1
        elif idx in (2, 3, 4):
            # Validate idx - 1 upcoming bytes
            if idx - 1 >= N - i:
                return False
            while idx - 1 > 0:
                i, idx = i + 1, idx - 1
                if not num2Bin(data[i]).startswith('10'):
                    return False
            i += 1
        else:
            return False

    return True

# Testing the solution
assert validUtf8([197, 130, 1]) == True
assert validUtf8([235, 140, 4]) == False
assert validUtf8([62]) == True
assert validUtf8([237,128,128]) == True
assert validUtf8([237,128]) == False